In [ ]:
import re  # 正则表达式提取文本
from jsonpath import jsonpath  # 解析json数据
import requests  # 发送请求
import pandas as pd  # 存取csv文件
import datetime  # 转换时间用

import time
import random


class Crawler:
    def __init__(self, id = 4860690891018271):
        self.id = id
        self.mid = id
        
        self.cookie = 'WEIBOCN_FROM=1110006030; SCF=At3cKMF8zNNlQvzopN0jE4GXnW6d1oMHWhTaIC30tCfqKKRqnDvCnrW-FINRVD2deutyyUpVAz_qetv-sFRd5J4.; SUB=_2A25O0rvLDeRhGeBP71YU8y7KzTuIHXVqPMWDrDV6PUJbktAKLVbCkW1NRUYLcSM1wE7gRu66wcI6nrX-vEo6cBuK; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFfYb7mkA6BHfpdMmuxGQ8k5JpX5K-hUgL.FoqpShBfe05cSoM2dJLoIE-LxKqLBoMLBo2LxKnL1KeL1-BLxKBLBo.L12zLxK-LB.qL1hSo; SSOLoginState=1675021211; ALF=1677613211; _T_WM=15194666543; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4860690891018271%26luicode%3D20000061%26lfid%3D4860690891018271%26uicode%3D20000061%26fid%3D4860690891018271; XSRF-TOKEN=926c8e; mweibo_short_token=a1ade2f776'

        self.url = f'https://m.weibo.cn/statuses/extend?id={self.id}'

        r = requests.get(self.url, timeout = 5)
        string = r.json()['data']['longTextContent']
        # df = pd.DataFrame(columns = ["index", "text"])
        # df = df.append({"index": id, "text": string}, ignore_index = True)
        df = pd.DataFrame(
            {
                "name": [''], 
                "粉丝数": [''], 
                "like_count": [''], 
                "source": [''], 
                "text": [string]
            }
        )
        
        df.to_csv("spider.csv", index = False)
    

    def sub_comment(self, comment_id):
        headers = {
            'Cookie': self.cookie,
            'Referer': 'https://m.weibo.cn/detail/' + self.id + '?cid=' + comment_id,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.61',
            'X-Requested-With': 'XMLHttpRequest'
        }

        # print(headers)

        times, max = 0, 1
        max_id, id_type = 0, 0
        var_index, var_text = [], []
        var_like_count = []
        var_name = []
        var_fans = []
        var_source = []

        while times < max:

            url = f'https://m.weibo.cn/comments/hotFlowChild?cid={comment_id}&max_id={max_id}&max_id_type={id_type}'
            print(url)
            r = requests.get(url, headers = headers, timeout = 5)

            if r.status_code == 200:
                cards = r.json()
                if cards.get('data') is not None:
                    cards = cards['data']

                    for i in cards:
                        # print(i['text'])
                        var_name.append(i['user']['screen_name'])
                        var_fans.append(i['user']['followers_count'])
                        var_text.append(i['text'])
                        var_like_count.append(i['like_count'])
                        var_source.append(i['source'])
                    
                    max_id = r.json()['max_id']
                    id_type = r.json()['max_id_type']
                    max = r.json()['max']

                    # time.sleep(1)
                    times += 1

                else:
                    print("return data is null")
                    break
        
        return [var_name, var_fans, var_text, var_like_count, var_source]
    

    def comment(self):
        headers = {
            'Cookie': self.cookie,
            'Referer': 'https://m.weibo.cn/detail/' + self.id,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.61',
            'X-Requested-With': 'XMLHttpRequest'
        }

        # print("main")
        # print(headers)
        times, max = 0, 1
        max_id, id_type = 0, 0

        while times < max:
            print(f'当前次数{times}为， 总共{max}次')
            url = f'https://m.weibo.cn/comments/hotflow?id={self.id}&mid={self.mid}&max_id={max_id}&max_id_type={id_type}' if max_id >= 1 else f'https://m.weibo.cn/comments/hotflow?id={self.id}&mid={self.mid}&max_id_type={id_type}'
            print(url)
            
            r = requests.get(url, headers = headers, timeout = 5)

            if r.status_code == 200:
                cards = r.json()
                if cards.get('data') is not None:
                    cards = r.json()['data']['data']
                    var_text = []
                    var_like_count = []
                    var_name = []
                    var_fans = []
                    var_source = []

                    time.sleep(random.uniform(0, 2))
                    times += 1 

                    # try:
                    for i in cards:
                        # print(i['id'], i['text'])

                        var_name.append(i['user']['screen_name'])
                        var_fans.append(i['user']['followers_count'])
                        var_text.append(i['text'])
                        var_like_count.append(i['like_count'])
                        var_source.append(i['source'])

                        return_var = self.sub_comment(i['id'])
                        var_name += return_var[0]
                        var_fans.extend(return_var[1])
                        var_text += return_var[2]
                        var_like_count.extend(return_var[3])
                        var_source += return_var[4]

                    df = pd.DataFrame(
                        {
                            "name": var_name, 
                            "粉丝数": var_fans,  
                            "like_count": var_like_count, 
                            "source": var_source, 
                            "text": var_text
                        }
                    )
                    # df = df.append({"index": cards['id'], "text": cards['text']}, ignore_index = True)
                    
                    df.to_csv("spider.csv", index = False, mode = 'a', header = False)

                    if times == 1: max = r.json()['data']['max']

                    max_id = r.json()['data']['max_id']
                    id_type = r.json()['data']['max_id_type']
                
                else:
                    print("data is null")
                    time.sleep(5)
                    continue

            # except Exception as e:
            #     print("max not found")
            #     print(e)
            #     continue
    
    def trans_time(self, v_str):
        """转换GMT时间为标准格式"""
        GMT_FORMAT = '%a %b %d %H:%M:%S +0800 %Y'
        timeArray = datetime.datetime.strptime(v_str, GMT_FORMAT)
        ret_time = timeArray.strftime("%Y-%m-%d %H:%M:%S")
        return ret_time

In [ ]:
if __name__ == '__main__':
    try:
        spider = Crawler('4860690891018271').comment()
    except KeyboardInterrupt:
        print("Program interrupted by user. Exiting...")
        del spider

In [5]:
import os
import re  # 正则表达式提取文本
from jsonpath import jsonpath  # 解析json数据
import requests  # 发送请求
import pandas as pd  # 存取csv文件
import datetime  # 转换时间用

import time
import re
import random

import urllib.parse

# https://m.weibo.cn/profile/1764661771

def getUserId(userName, pageNum):
    name = urllib.parse.quote('=1&q=' + userName)
    url = f'https://m.weibo.cn/api/container/getIndex?containerid=100103type{name}&page_type=searchall'

    # print('https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%E6%B8%85%E5%8D%8E%E5%A4%A7%E5%AD%A6&page_type=searchall')
    # print(url)

    headers = {
    'Cookie': '_T_WM=69514023915; MLOGIN=1; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFfYb7mkA6BHfpdMmuxGQ8k5JpX5K-hUgL.FoqpShBfe05cSoM2dJLoIE-LxKqLBoMLBo2LxKnL1KeL1-BLxKBLBo.L12zLxK-LB.qL1hSo; SCF=At3cKMF8zNNlQvzopN0jE4GXnW6d1oMHWhTaIC30tCfq6k41yT5HWkNhZ5TIvDrF5_t-RjdCZ-1J3mknscKAxjU.; SUB=_2A25O3rybDeRhGeBP71YU8y7KzTuIHXVqIMTTrDV6PUJbktANLWfxkW1NRUYLcUYi8i0Y-SvhDTT3Qd54PBXW9h3c; SSOLoginState=1675283659; ALF=1677875659; WEIBOCN_FROM=1110106030; XSRF-TOKEN=1293ca; mweibo_short_token=60cc77794f; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D231051_-_followers_-_1764661771%26fid%3D231051_-_followers_-_1764661771%26uicode%3D10000011',
    'Referer': 'https://m.weibo.cn/search?containerid=100103type' + name,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.61',
    'X-Requested-With': 'XMLHttpRequest'
    }

    r = requests.get(url, headers = headers, timeout = 5)
    # print(r.json())
    uid = r.json()['data']['cards'][0]['card_group'][0]['user']['id']
    return uid

def scoll_search():
    pass


uni_list = ['北京大学','清华大学','北京师范大学','北京外国语大学','北京交通大学','北京邮电大学','中国政法大学','中央财经大学','中央美术学院','对外经济贸易大学','南开大学','天津大学','大连理工大学','东北大学','吉林大学','东北师范大学','复旦大学','上海交通大学','同济大学','华东理工大学','东华大学','华东师范大学','上海外国语大学','南京大学','东南大学','河海大学','江南大学','南京农业大学','浙江大学','厦门大学','山东大学','中国海洋大学','中国石油大学（华东)','武汉大学','华中科技大学','中国地质大学（武汉)','武汉理工大学','华中农业大学','中南财经政法大学','湖南大学','中南大学','中山大学','重庆大学','西南大学','西南交通大学','西安交通大学','西北农林科技大学','陕西师范大学','西安电子科技大学','兰州大学','哈尔滨工业大学','哈尔滨工程大学','南京航空航天大学','中国科学技术大学','北京电影学院','辽宁大学','上海大学','南京林业大学','浙江工商大学','安徽大学','河南大学','武汉纺织大学','广西大学','西南政法大学','四川师范大学','贵州大学']

if __name__ == "__main__":
    for i in uni_list:
        try:
            print(i, getUserId(i, "1"))
        except:
            print(i, "not found")
            continue

北京大学 3237705130
清华大学 1676317545
北京师范大学 1875088617
北京外国语大学 1873032605
北京交通大学 1865345137
北京邮电大学 1844283341
中国政法大学 1853914071
中央财经大学 3113890133
中央美术学院 1998766813
对外经济贸易大学 1980761473
南开大学 1851774145
天津大学 1871729063
大连理工大学 3082822153
东北大学 1891849065
吉林大学 2271848313
东北师范大学 1924502695
复旦大学 1729332983
上海交通大学 1739746697
同济大学 1865154537
华东理工大学 2028734565
东华大学 1880834235
华东师范大学 1688511087
上海外国语大学 1887685537
东南大学 1703010470
河海大学 1853565117
江南大学 1854563221
南京农业大学 1905782710
浙江大学 1851755225
厦门大学 1879192935
山东大学 1880883723
中国海洋大学 1825107551
中国石油大学（华东) 1865978564
武汉大学 1666177401
华中科技大学 1663414103
武汉理工大学 1899783701
华中农业大学 1690023261
中南财经政法大学 1216903164
湖南大学 1851726313
中南大学 1871765890
中山大学 1892723783
重庆大学 1875333245
西南大学 1973665271
西南交通大学 2418433987
西安交通大学 1862928653
西北农林科技大学 3204433793
陕西师范大学 1833031745
西安电子科技大学 1854580681
兰州大学 1878134457
哈尔滨工业大学 1873625985
哈尔滨工程大学 1865804041
南京航空航天大学 1805101535
中国科学技术大学 1823887605
北京电影学院 1875379117
辽宁大学 1898999173
上海大学 3243026514
南京林业大学 2237972353
浙江工商大学 3317008062
安徽

### 草稿

In [ ]:
# pattern = "[\u4e00-\u9fa5]{5}"

# matches = re.findall(pattern, string)

# if matches:
#     for match in matches:
#         print(match, end = "")
# else:
#     print("匹配失败")

# except:
# 		continue

# finally:
# 	time.sleep(random.uniform(0, 2))
# 	times += 1

# # 转发数
# reposts_count_list = jsonpath(cards, '$..mblog.reposts_count')
# # 评论数
# comments_count_list = jsonpath(cards, '$..mblog.comments_count')
# # 点赞数
# attitudes_count_list = jsonpath(cards, '$..mblog.attitudes_count')

# # 请求地址
# url = 'https://m.weibo.cn/api/container/getIndex'
# # 请求参数
# params = {
# 	"containerid": "100103type=1&q={}".format(topic),
# 	"page_type": "searchall",
# 	# "page": searchall
# }
# r = requests.get(url, headers = headers, params = params)

In [ ]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

class Crawler:

    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls

    def download_url(self, url):
        return requests.get(url).text

    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path

    def add_url_to_visit(self, url):
        if url not in self.visited_urls and url not in self.urls_to_visit:
            self.urls_to_visit.append(url)

    def crawl(self, url):
        html = self.download_url(url)
        for url in self.get_linked_urls(url, html):
            self.add_url_to_visit(url)

    def run(self):
        while self.urls_to_visit:
            url = self.urls_to_visit.pop(0)
            logging.info(f'Crawling: {url}')
            try:
                self.crawl(url)
            except Exception:
                logging.exception(f'Failed to crawl: {url}')
            finally:
                self.visited_urls.append(url)

if __name__ == '__main__':
    Crawler(urls=['https://www.imdb.com/']).run()